# Traveling Salesman Problem (TSP)

**Objectives**

- Introduce students to a real world problem solved by OR practitioners
- Demonstrate the use of heuristics to obtain good solutions to optimization problems
- Give students an appreciation of the difficulty of solving optimization problems exactly

**Reading:** Read Handout 2 on the traveling salesman problem.

**Brief description:** Finding an optimal solution to a Traveling Salesman Problem, and proving that it is, in fact, an optimal solution, is a difficult task. In practice, when a feasible solution to a difficult problem needs to be provided quickly, one often resorts to using heuristics, i.e., procedures for generating feasible solutions, or improving existing ones, that can be executed quickly and, hopefully, produce a pretty good result. In this lab we will consider several such heuristic procedures for the TSP.

<font color='blue'> <b>Solutions are shown blue.</b> </font> <br>
<font color='red'> <b>Instuctor comments are shown in red.</b> </font>

<font color='red'>A tool we might want to use: [TSP DIY](https://www.math.uwaterloo.ca/tsp/app/diy.html)  </font>

<font color='red'> Some explanation of what imports are here.</font>

In [46]:
# imports -- don't forget to run this cell
import numpy as np
import math
import pandas as pd
from random import randrange

In [30]:
def tsp_grid_instance(n, m, manhattan=True):
    """Return a distance matrix (manhattan or euclidian) on an n*m grid.
    
    Args:
        n (int): width of the grid.
        m (int): height of the grid.
        manhattan (bool): return manhattan distance matrix if true. Otherwise, return euclidian.
    """
    # create nodes of grid
    nodes = []
    for i in range(n):
        for j in range(m):
            nodes.append((i,j))

    # create distance matrix
    d = np.zeros((len(nodes),len(nodes)))
    for i in range(len(nodes)):
        for j in range(len(nodes)):
            if manhattan:
                d[i][j] = abs(nodes[i][0] - nodes[j][0]) + abs(nodes[i][1] - nodes[j][1])
            else:
                d[i][j] = math.sqrt((nodes[i][0] - nodes[j][0])**2 + (nodes[i][1] - nodes[j][1])**2)
    
    return d

In [83]:
def tour_cost(G, tour):
    """Return the cost of the tour on graph G.
    
    Args:
        G (np.ndarray): adjacency matrix representing a graph.
        tour (List[int]): ordered list of nodes visited on the tour.
    """
    return sum([G[tour[i],tour[i+1]] for i in range(len(tour)-1)])

In [434]:
def neighbor(G, initial, nearest):
    """Run a neighbor heuristic on G starting at the given initial node.
    
    Args:
        G (np.ndarray): adjacency matrix representing a graph.
        intial (int): index of the node to start from.    
        nearest (bool): run nearest neighbor if true. Otherwise, run random.
    """
    unvisited = list(range(len(G))) # list of nodes
    
    # start tour at initial and remove it from unvisited
    tour = [initial]
    unvisited.remove(initial) 
    
    # choose next node from unvisited
    while len(unvisited) > 0:
        if nearest:
            # dictionary from univisited nodes to their distance from current node
            d = {i : G[tour[-1]][i] for i in range(len(G[tour[-1]])) if i in unvisited}
            # randomly select next node among the nearest
            min_val = min(d.values())
            possible = [k for k, v in d.items() if v==min_val]
            next_node = possible[randrange(len(possible))]
        else:
            next_node = unvisited[randrange(len(unvisited))]
        tour.append(next_node)
        unvisited.remove(next_node) 
        
    # go back to start
    tour.append(initial)
    
    return tour

In [435]:
def random_neighbor(G, initial=0):
    """Run the nearest neighbor heuristic on G starting at the given initial node.
    
    Args:
        G (np.ndarray): adjacency matrix representing a graph.
        intial (int): index of the node to start from.    
    """
    return neighbor(G, initial, False)

def nearest_neighbor(G, initial=0):
    """Run the nearest neighbor heuristic on G starting at the given initial node.
    
    Args:
        G (np.ndarray): adjacency matrix representing a graph.
        intial (int): index of the node to start from.    
    """
    return neighbor(G, initial, True)

In [436]:
def insertion(G, initial, nearest):
    """Run an insertion heuristic on G starting with the given initial 2-node tour."""
    
    unvisited = list(range(len(G))) # list of nodes
    
    # start tour at initial and remove it from unvisited
    tour = list(initial)
    unvisited.remove(initial[0]) 
    unvisited.remove(initial[1]) 
    
    # choose next node from unvisited
    while len(unvisited) > 0:
        # dictionary from univisited nodes to their shortest distance from tour
        d = G[:,[0,1]].min(axis=1)
        d = {i : d[i] for i in range(len(d)) if i in unvisited}
        if nearest:
            min_val = min(d.values())
            possible = [k for k, v in d.items() if v==min_val]    
        else:
            max_val = max(d.values())
            possible = [k for k, v in d.items() if v==max_val]    
        next_node = possible[randrange(len(possible))]
        
        # insert node into tour at minimum cost
        increase = [G[tour[i], next_node]
                    + G[next_node, tour[i+1]] 
                    - G[tour[i], tour[i+1]] for i in range(len(tour)-1)]
        insert_index = increase.index(min(increase))+1
        tour.insert(insert_index, next_node)
        unvisited.remove(next_node) 
    
    return tour

In [441]:
def nearest_insertion(G, initial=[0,1,0]):
    """Run the nearest insertion heuristic on G starting with the given initial 2-node tour.
    
    Args:
        G (np.ndarray): adjacency matrix representing a graph.
        intial (List[int]): initial 2-node tour. 
    """
    return insertion(G, initial, True)

def furthest_insertion(G, initial=[0,len(G)-1,0]):
    """Run the furthest insertion heuristic on G starting with the given initial 2-node tour.
    
    Args:
        G (np.ndarray): adjacency matrix representing a graph.
        intial (int): initial 2-node tour.  
    """
    return insertion(G, initial, False)

In [446]:
G = tsp_grid_instance(6,8,True)
tour = random_neighbor(G)
tour = nearest_neighbor(G)
tour = nearest_insertion(G)
tour = furthest_insertion(G)
tour_cost(G, tour)

58.0

In [447]:
n = 250 
rand_n = 0
near_n = 0
near_ins = 0
furth_ins = 0
for i in range(n):
    G = tsp_grid_instance(6,8,True)
    rand_n += tour_cost(G, random_neighbor(G))
    near_n += tour_cost(G, nearest_neighbor(G))
    near_ins += tour_cost(G, nearest_insertion(G))
    furth_ins += tour_cost(G, furthest_insertion(G))
rand_n /= n
near_n /= n
near_ins /= n
furth_ins /= n
print(rand_n, near_n, near_ins, furth_ins)

223.784 63.536 56.32 56.008
